#SQLCoder-7b-2
Run the cells below to run inference on our text-to-SQL LLM: SQLCoder-7b-2.

⭐️ [Github Repo](https://github.com/defog-ai/sqlcoder)

🤗 [Huggingface Page](https://huggingface.co/defog/sqlcoder-7b-2)

## Init

Add HF_TOEKN, ngrok_token to the secret tab in google collab from huggingface and ngrok

##Setup

In [ ]:
!pip install torch transformers bitsandbytes accelerate sqlparse pyngrok flask llama-index llama-index-llms-huggingface llama-index-embeddings-huggingface llama-index-readers-web

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 8.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-many

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip freeze > drive/MyDrive/Augment/requirements2.txt

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

##Download the Model
Use any model on Colab (or any system with >30GB VRAM on your own machine) to load this in f16. If unavailable, use a GPU with minimum 8GB VRAM to load this in 8bit, or with minimum 5GB of VRAM to load in 4bit.

This step can take around 5 minutes

In [ ]:
# Load defogSQLCoder from HuggingFace

def load_model(model_name):
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = AutoModelForCausalLM.from_pretrained(
          model_name,
          trust_remote_code=True,
          torch_dtype=torch.float16,
          load_in_4bit=True,
          device_map="auto",
          use_cache=True,
      )
  return model, tokenizer

##Set the Question & Prompt and Tokenize
Feel free to change the schema in the prompt below to your own schema

In [ ]:
# Read database schema, made adjustment to schema, removed pl/SQL syntax and added relationship manually for prompt enbgineering
def read_schema(path):
  # Open the file in read mode
  with open(path, 'r') as file:
      # Read the entire file into the string variable
      file_contents = file.read()

  # Now file_contents contains the entire content of the file
  return file_contents

In [ ]:
# Create prompt
def build_prompt(schema):
  prompt_task = """
  ### Task
  Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

  """
  prompt_instructions = """
  ### Instructions
  - If you cannot answer the question with the available database schema, return 'I do not know'
  """

  prompt_db = """
  ### Database Schema
  This query will run on a database whose schema is represented in this string:
  """ + schema

  prompt_answer = """
  ### Answer
  Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
  [SQL]
  """
  prompt = prompt_task + prompt_instructions + prompt_db + prompt_answer
  return prompt



##Generate the SQL
This can be excruciatingly slow on a T4 in Colab, and can take 10-20 seconds per query. On faster GPUs, this will take ~1-2 seconds

Ideally, you should use `num_beams`=4 for best results. But because of memory constraints, we will stick to just 1 for now.

In [ ]:
import sqlparse
# Inference, text to sql
def generate_query(question, prompt, tokenizer, model):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=3900,
        do_sample=False,
        num_beams=1
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

In [ ]:
def load_pipeline(model_name, schema_path):
  model, tokenizer = load_model(model_name)
  schema = read_schema(schema_path)
  prompt = build_prompt(schema)
  return model, tokenizer, prompt

In [ ]:
def inference_pipeline(model, tokenizer, prompt,question):
  output = generate_query(question, prompt, tokenizer, model)
  return output

In [ ]:
# Load chat Model llama3
def llm_2():
  import torch
  from transformers import BitsAndBytesConfig
  from llama_index.core.prompts import PromptTemplate
  from llama_index.llms.huggingface import HuggingFaceLLM

  quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_compute_dtype=torch.float16,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_use_double_quant=True,
  )
  llm = HuggingFaceLLM(
    model_name="meta-llama/Meta-Llama-3-8B-Instruct",
    tokenizer_name="meta-llama/Meta-Llama-3-8B-Instruct",
    context_window=3900,
    max_new_tokens=3900,
    model_kwargs={"quantization_config": quantization_config},
    # tokenizer_kwargs={},
    device_map="auto",
)
  return llm

In [ ]:
import getpass
import os
import threading
from flask import Flask,request, jsonify,current_app
from pyngrok import ngrok, conf
from google.colab import userdata



print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken")
conf.get_default().auth_token =userdata.get('ngrok_token')

app = Flask(__name__)

# Open a ngrok tunnel to the HTTP server
domain = "ideal-amoeba-specially.ngrok-free.app"
public_url = ngrok.connect(4000, domain="ideal-amoeba-specially.ngrok-free.app").public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, 4000))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# ... Update inbound traffic via APIs to use the public-facing ngrok URL
def initialize_model(model_name, schema_path):
    # Initialize both models
    print(" ---- Text to SQL Model --- ")
    model, tokenizer, prompt = load_pipeline(model_name, schema_path)
    print(" ----- Chat Model -----")
    llm = llm_2()
    return model, tokenizer, prompt, llm

@app.route("/load")
def init_model():
    model_name = "defog/sqlcoder-7b-2"
    schema_path = "drive/MyDrive/Augment/schema.txt"

    # Ensure set global variables if empty both models, tokenizer or prompt
    if not current_app.config.get('model') or \
       not current_app.config.get('tokenizer') or \
       not current_app.config.get('prompt'):
        print("loading, model")
        model, tokenizer, prompt, llm = initialize_model(model_name, schema_path)
        current_app.config['model'] = model
        current_app.config['tokenizer'] = tokenizer
        current_app.config['prompt'] = prompt
        current_app.config['llm'] = llm
        print("----- model complete -----")
        return {"message": "Model Loaded Succesfully"}
    print("model already loaded")
    return {"message": "Model Already Loaded"}

# Agent logic
"""
Ask to llm chat if the query can be converted to SQL,
if not continue with the chat model
if yes go to defogSQLCoder and convert text to sql
"""
@app.route("/query", methods=["POST"])
def text_to_sql():
    from llama_index.core.llms import ChatMessage
    import re
    input_data = request.json.get("input")
    messages = [
    ChatMessage(role="system", content="You are a logical assistant that answer only with yes or no"),
    ChatMessage(role="user", content=f"does it make sense to convert '{input_data}' to an SQL query ?"),
    ]
    # Post Processing
    result = str(current_app.config['llm'].chat(messages)).replace("assistant: assistant","").lower()
    print(result)
    if re.search(r'\bno\b', result.strip()):
      messages = [
      ChatMessage(role="system", content="You are an HTML expert that will format the response accordingly"),
      ChatMessage(role="user", content=f"{input_data} "),
      ]
      text = str(current_app.config['llm'].chat(messages)).replace("assistant: assistant","")
      return({"query":text})
    # Access query_engine and llm from the application context
    else:
      print("yes it can be converted")
      input_data = input_data + ", extract the information only for the user first_name=Patrick last_name=Saade"
      result = inference_pipeline(current_app.config['model'], current_app.config['tokenizer'],current_app.config['prompt'], input_data)
      return {"query":result}



# Start the Flask server in a new thread
app.run(port=4000)

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken
 * ngrok tunnel "https://ideal-amoeba-specially.ngrok-free.app" -> "http://127.0.0.1:4000/"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:4000
INFO:werkzeug:Press CTRL+C to quit


loading, model
 ---- Text to SQL Model --- 


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

...

INFO:werkzeug:127.0.0.1 - - [21/May/2024 14:34:55] "POST /query HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




no


INFO:werkzeug:127.0.0.1 - - [21/May/2024 14:35:57] "POST /query HTTP/1.1" 200 -
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




yes
yes it can be converted


/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [21/May/2024 14:37:02] "POST /query HTTP/1.1" 200 -
